In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report



# 1. 데이터 불러오기

In [2]:
df = pd.read_csv("./data/titanic.csv")

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
prep_df = df.copy()
prep_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    str    
 4   Sex          891 non-null    str    
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    str    
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    str    
 11  Embarked     889 non-null    str    
dtypes: float64(2), int64(5), str(5)
memory usage: 83.7 KB


# 2. 결측치 및 이상치 처리

In [4]:
prep_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    str    
 4   Sex          891 non-null    str    
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    str    
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    str    
 11  Embarked     889 non-null    str    
dtypes: float64(2), int64(5), str(5)
memory usage: 83.7 KB


In [5]:
print(prep_df["Age"].mean())
print(prep_df["Age"].median())
print(prep_df["Age"].mode())

29.69911764705882
28.0
0    24.0
Name: Age, dtype: float64


In [6]:
# 나이 결측치 median 값으로 채우기 - 중수
#prep_df["Age"] = prep_df["Age"].fillna(df["Age"].median())
#prep_df.info()

In [7]:
# 각 Pclass별로 median값으로 나이 결측치 채우기
prep_df['Age'] = (
    prep_df
    .groupby(['Pclass','Sex'])['Age']
    .transform(lambda x: x.fillna(x.median()))
    .astype(int)
)
prep_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    str    
 4   Sex          891 non-null    str    
 5   Age          891 non-null    int64  
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    str    
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    str    
 11  Embarked     889 non-null    str    
dtypes: float64(1), int64(6), str(5)
memory usage: 83.7 KB


In [8]:
prep_df["Embarked"].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [9]:
prep_df["Embarked"] = prep_df["Embarked"].fillna("N")
prep_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    str    
 4   Sex          891 non-null    str    
 5   Age          891 non-null    int64  
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    str    
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    str    
 11  Embarked     891 non-null    str    
dtypes: float64(1), int64(6), str(5)
memory usage: 83.7 KB


In [10]:
prep_df["Cabin"].str[0].value_counts()

Cabin
C    59
B    47
D    33
E    32
A    15
F    13
G     4
T     1
Name: count, dtype: int64

In [11]:
prep_df["Cabin_initial"] = prep_df["Cabin"].str[0]

In [12]:
prep_df["Survived"].value_counts()

Survived
0    549
1    342
Name: count, dtype: int64

In [13]:
mask = prep_df["Cabin_initial"].notna()
prep_df.loc[mask,"Survived"].value_counts()

Survived
1    136
0     68
Name: count, dtype: int64

In [14]:
survival_by_cabin = prep_df.groupby("Cabin_initial")["Survived"].mean()
survival_by_cabin

Cabin_initial
A    0.466667
B    0.744681
C    0.593220
D    0.757576
E    0.750000
F    0.615385
G    0.500000
T    0.000000
Name: Survived, dtype: float64

In [15]:
prep_df["Pclass"].value_counts()

Pclass
3    491
1    216
2    184
Name: count, dtype: int64

In [16]:
survival_by_Pcls = prep_df.groupby("Pclass")["Survived"].mean()
survival_by_Pcls

Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64

In [17]:
survival_by_sex = prep_df.groupby("Sex")["Survived"].mean()
survival_by_sex

Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64

In [18]:
prep_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PassengerId    891 non-null    int64  
 1   Survived       891 non-null    int64  
 2   Pclass         891 non-null    int64  
 3   Name           891 non-null    str    
 4   Sex            891 non-null    str    
 5   Age            891 non-null    int64  
 6   SibSp          891 non-null    int64  
 7   Parch          891 non-null    int64  
 8   Ticket         891 non-null    str    
 9   Fare           891 non-null    float64
 10  Cabin          204 non-null    str    
 11  Embarked       891 non-null    str    
 12  Cabin_initial  204 non-null    str    
dtypes: float64(1), int64(6), str(6)
memory usage: 90.6 KB


In [19]:
survival_by_embark = prep_df.groupby("Embarked")["Survived"].mean()
survival_by_embark

Embarked
C    0.553571
N    1.000000
Q    0.389610
S    0.336957
Name: Survived, dtype: float64

In [20]:
prep_df = pd.get_dummies(prep_df, columns=["Embarked"], drop_first=False)

In [21]:
prep_df["Cabin_known"] = prep_df["Cabin_initial"].notna().astype(int)

prep_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Cabin_initial,Embarked_C,Embarked_N,Embarked_Q,Embarked_S,Cabin_known
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,NaN,False,False,False,True,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,True,False,False,False,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,NaN,False,False,False,True,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,C,False,False,False,True,1
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,NaN,False,False,False,True,0


In [22]:
prep_df["Cabin_initial"] = prep_df["Cabin_initial"].fillna("Unknown")
prep_df = pd.get_dummies(prep_df, columns=["Cabin_initial"], drop_first=False)
prep_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Cabin_known,Cabin_initial_A,Cabin_initial_B,Cabin_initial_C,Cabin_initial_D,Cabin_initial_E,Cabin_initial_F,Cabin_initial_G,Cabin_initial_T,Cabin_initial_Unknown
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,...,0,False,False,False,False,False,False,False,False,True
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,...,1,False,False,True,False,False,False,False,False,False
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,...,0,False,False,False,False,False,False,False,False,True
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,...,1,False,False,True,False,False,False,False,False,False
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,...,0,False,False,False,False,False,False,False,False,True


In [23]:
prep_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PassengerId            891 non-null    int64  
 1   Survived               891 non-null    int64  
 2   Pclass                 891 non-null    int64  
 3   Name                   891 non-null    str    
 4   Sex                    891 non-null    str    
 5   Age                    891 non-null    int64  
 6   SibSp                  891 non-null    int64  
 7   Parch                  891 non-null    int64  
 8   Ticket                 891 non-null    str    
 9   Fare                   891 non-null    float64
 10  Cabin                  204 non-null    str    
 11  Embarked_C             891 non-null    bool   
 12  Embarked_N             891 non-null    bool   
 13  Embarked_Q             891 non-null    bool   
 14  Embarked_S             891 non-null    bool   
 15  Cabin_known      

In [24]:
prep_df["FamilySize"] = prep_df["SibSp"] + prep_df["Parch"] + 1
prep_df["IsAlone"] = (prep_df["FamilySize"] == 1).astype(int)
prep_df["Title"] = df["Name"].str.extract(r" ([A-Za-z]+)\.", expand=False)
prep_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Cabin_initial_C,Cabin_initial_D,Cabin_initial_E,Cabin_initial_F,Cabin_initial_G,Cabin_initial_T,Cabin_initial_Unknown,FamilySize,IsAlone,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,...,False,False,False,False,False,False,True,2,0,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,...,True,False,False,False,False,False,False,2,0,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,...,False,False,False,False,False,False,True,1,1,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,...,True,False,False,False,False,False,False,2,0,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,...,False,False,False,False,False,False,True,1,1,Mr


In [25]:
prep_df["Title"].value_counts()

Title
Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Major         2
Mlle          2
Col           2
Don           1
Mme           1
Ms            1
Lady          1
Sir           1
Capt          1
Countess      1
Jonkheer      1
Name: count, dtype: int64

In [26]:
cols_to_drop = ["Name", "Ticket", "PassengerId", "Cabin", "SibSp", "Parch"]

prep_df = prep_df.drop(cols_to_drop, axis=1)

In [27]:
encoder = LabelEncoder()

prep_df["Title"] = encoder.fit_transform(prep_df["Title"])
prep_df["Title"].head()

0    12
1    13
2     9
3    13
4    12
Name: Title, dtype: int64

In [28]:
encoder = LabelEncoder()
prep_df["Sex"] = encoder.fit_transform(prep_df["Sex"])
encoder.classes_

array(['female', 'male'], dtype=object)

In [29]:
prep_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked_C,Embarked_N,Embarked_Q,Embarked_S,Cabin_known,...,Cabin_initial_C,Cabin_initial_D,Cabin_initial_E,Cabin_initial_F,Cabin_initial_G,Cabin_initial_T,Cabin_initial_Unknown,FamilySize,IsAlone,Title
0,0,3,1,22,7.2500,False,False,False,True,0,...,False,False,False,False,False,False,True,2,0,12
1,1,1,0,38,71.2833,True,False,False,False,1,...,True,False,False,False,False,False,False,2,0,13
2,1,3,0,26,7.9250,False,False,False,True,0,...,False,False,False,False,False,False,True,1,1,9
3,1,1,0,35,53.1000,False,False,False,True,1,...,True,False,False,False,False,False,False,2,0,13
4,0,3,1,35,8.0500,False,False,False,True,0,...,False,False,False,False,False,False,True,1,1,12


In [30]:
X = prep_df.drop("Survived", axis=1)
y = prep_df["Survived"]

In [31]:
X["Pclass"] = X["Pclass"].astype(int)
X.info()

<class 'pandas.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Pclass                 891 non-null    int64  
 1   Sex                    891 non-null    int64  
 2   Age                    891 non-null    int64  
 3   Fare                   891 non-null    float64
 4   Embarked_C             891 non-null    bool   
 5   Embarked_N             891 non-null    bool   
 6   Embarked_Q             891 non-null    bool   
 7   Embarked_S             891 non-null    bool   
 8   Cabin_known            891 non-null    int64  
 9   Cabin_initial_A        891 non-null    bool   
 10  Cabin_initial_B        891 non-null    bool   
 11  Cabin_initial_C        891 non-null    bool   
 12  Cabin_initial_D        891 non-null    bool   
 13  Cabin_initial_E        891 non-null    bool   
 14  Cabin_initial_F        891 non-null    bool   
 15  Cabin_initial_G  

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3333, stratify=y)

In [33]:
X_train

,Pclass,Sex,Age,Fare,Embarked_C,Embarked_N,Embarked_Q,Embarked_S,Cabin_known,Cabin_initial_A,...,Cabin_initial_C,Cabin_initial_D,Cabin_initial_E,Cabin_initial_F,Cabin_initial_G,Cabin_initial_T,Cabin_initial_Unknown,FamilySize,IsAlone,Title
56,2,0,21,10.5000,False,False,False,True,0,False,...,False,False,False,False,False,False,True,1,1,9
322,2,0,30,12.3500,False,False,True,False,0,False,...,False,False,False,False,False,False,True,1,1,9
201,3,1,25,69.5500,False,False,False,True,0,False,...,False,False,False,False,False,False,True,11,0,12
760,3,1,25,14.5000,False,False,False,True,0,False,...,False,False,False,False,False,False,True,1,1,12
445,1,1,4,81.8583,False,False,False,True,1,True,...,False,False,False,False,False,False,False,3,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,1,0,50,28.7125,True,False,False,False,1,False,...,True,False,False,False,False,False,False,1,1,9
39,3,0,14,11.2417,True,False,False,False,0,False,...,False,False,False,False,False,False,True,2,0,9
815,1,1,40,0.0000,False,False,False,True,1,False,...,False,False,False,False,False,False,False,1,1,12
631,3,1,51,7.0542,False,False,False,True,0,False,...,False,False,False,False,False,False,True,1,1,12


In [34]:
scaler = StandardScaler()

scale_cols = ["Age", "Fare", "FamilySize"]

X_train[scale_cols] = scaler.fit_transform(X_train[scale_cols])

X_test[scale_cols] = scaler.transform(X_test[scale_cols])

In [35]:
print(X_train[scale_cols].mean().round(2))
print(X_train[scale_cols].std().round(2))

Age           0.0
Fare         -0.0
FamilySize   -0.0
dtype: float64
Age           1.0
Fare          1.0
FamilySize    1.0
dtype: float64


In [ ]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(
    n_estimators=1000,
    random_state=3333,
    learning_rate=1e-2,
    max_depth=5,
    eval_metric="logloss"
)

xgb_model.fit(X_train, y_train)

pred = xgb_model.predict(X_test)
res = np.equal(y_test, pred).sum()/pred.size
res

np.float64(0.8435754189944135)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#model = DecisionTreeClassifier(random_state=3333)
model = RandomForestClassifier(n_estimators=300, random_state=3333)


params = {"max_depth": [15,16,17,18,19,20], "min_samples_split": [6,7,8,9,10]}

grid_model = GridSearchCV(model, param_grid=params, cv=5)

grid_model.fit(X_train, y_train)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",RandomForestC...om_state=3333)
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'max_depth': [15, 16, ...], 'min_samples_split': [6, 7, ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also displayed;

In [ ]:
print(grid_model.best_score_)
grid_model.best_params_

0.8342263370432385


{'max_depth': 17, 'min_samples_split': 9}

In [ ]:
pred = grid_model.best_estimator_.predict(X_test)

res = np.equal(y_test, pred).sum()/pred.size
res

np.float64(0.8435754189944135)

In [ ]:
y_train.head()

56     1
322    1
201    0
760    0
445    1
Name: Survived, dtype: int64